In [9]:
id: str = 'rise-event'
spektrix_event_ids: str = '4801ANHPDLLRCQPPBDBQTRQJMPNKSLKPJ'
venue_postcode: str = 'BD1 1LA'
rosterfy_event_ids: str = '25890'
project_id: str = 'recf4tPyjoXi81NDo'

dates = ['2025-01-10 18:30:00', '2025-01-11 18:30:00']

In [10]:
from datetime import datetime

import petl as etl

from utils.process.event import Tickets, Volunteers, Sustainability

from utils.paths import PUBLISHED, SITE

In [11]:
OUTPUT_DIR = SITE / 'events' / id / '_data/event'
TICKETS_DIR = OUTPUT_DIR / 'tickets'
TICKETS_DIR.mkdir(exist_ok=True, parents=True)
VOLUNTEERS_DIR = OUTPUT_DIR / 'volunteers'
SUSTAINABILITY_DIR = OUTPUT_DIR / 'sustainability'

In [12]:
event_times = [datetime.fromisoformat(d) for d in dates]

In [13]:
if spektrix_event_ids:
    tickets = Tickets(spektrix_event_ids.split())
    tickets.detailed().tocsv(TICKETS_DIR / 'by_geography.csv')
    tickets.summarise().tocsv(TICKETS_DIR / 'summary.csv')
    tickets.types().tocsv(TICKETS_DIR / 'by_type.csv')

In [14]:
if rosterfy_event_ids:
    VOLUNTEERS_DIR.mkdir(exist_ok=True, parents=True)
    volunteers = Volunteers(str(rosterfy_event_ids).split())
    volunteers.summarise().tocsv(VOLUNTEERS_DIR / 'summary.csv')

In [28]:
if project_id:
    SUSTAINABILITY_DIR.mkdir(exist_ok=True, parents=True)
    sustainability = Sustainability(project_id).summarise()
    sustainability.tocsv(SUSTAINABILITY_DIR / 'breakdown.csv')
    sustainability.select(lambda r: r.impact_type == 'ALL' and r.scope == 'ALL').cut('tCO2e').tojson(SUSTAINABILITY_DIR / 'summary.json', lines=True)